---
title: Migration Graphs and Spatial Interaction
jupyter: conda-env-nersa25-py
---

In [ ]:
import geopandas as gpd
import numpy as np
import networkx as nx
import pandas as pd
import statsmodels.formula.api as smf
from formulaic import Formula
from geosnap import DataStore
from geosnap import io as gio
from libpysal.graph import Graph
from lonboard import Map, PolygonLayer, ScatterplotLayer, basemap, viz
from lonboard.experimental import ArcLayer
from lonboard.layer_extension import BrushingExtension
from mapclassify.util import get_color_array
from spint.gravity import Attraction, Doubly, Gravity, Production
from statsmodels.api import families
%load_ext watermark
%watermark -a 'eli knaap'

## Demographic & Socioeconomic Data

### ACS Data

In [ ]:
datasets = DataStore()

dc = gio.get_acs(datasets, state_fips="11", years=2021, level="tract")

In [ ]:
dc.head()

In [ ]:
dc.plot()

### LODES Data

In [ ]:
dc_flows = pd.read_csv(
    "https://lehd.ces.census.gov/data/lodes/LODES8/dc/od/dc_od_main_JT00_2022.csv.gz",
    converters={"w_geocode": str, "h_geocode": str},
    low_memory=False,
    encoding="latin1",
)

In [ ]:
dc_flows

add tract-level geoids and drop extraneous columns

In [ ]:
dc_flows["w_tr_geocode"] = dc_flows["w_geocode"].str[:11]
dc_flows["h_tr_geocode"] = dc_flows["h_geocode"].str[:11]

dc_flows = dc_flows[["w_geocode", "h_geocode", "w_tr_geocode", "h_tr_geocode", "S000"]]

pretend like its a closed system, only consdidering intra-regional flows

aggregate to tract-level flows (from block-level)

In [ ]:
dc_flows = (
    dc_flows.groupby(["w_tr_geocode", "h_tr_geocode"])["S000"].sum().reset_index()
)

this is an origin-destination matrix (as an adjacency list). `S000` is the number of flows between home census tract `h_tr_geocode` and work tract `w_tr_geocode`. There are many intra-tract flows $W_{ii}\neq0$} (people commuting within the same tract, or *possibly* WFH, though that would be tough with LODES) and the flows are *directed* $W_{ij} \neq W_{ji}$

In [ ]:
dc_flows

create a pysal Graph to represent these flows. to represent the direction of flow correctly the focal column is home and neighbor is work (this shows the AM commute)

In [ ]:
dc_flow_graph = Graph.from_adjacency(
    adjacency=dc_flows,
    focal_col="h_tr_geocode",
    neighbor_col="w_tr_geocode",
    weight_col="S000",
)

In [ ]:
dc_flow_graph.summary()

In [ ]:
dc_flow_graph.adjacency.head()

the `Graph` is ordered, and to visualize it correctly we want to align our other data to match this Graph; so stash the id/order

In [ ]:
idx = dc_flow_graph.unique_ids

create a geodataframe of centroids to visualize point-to-point flows, then reindex to re-order appropriately and drop observations with no flow.

In [ ]:
dc_centroids = dc.set_geometry(dc.centroid)
dc_centroids = dc_centroids.set_index("geoid").reindex(idx)

In [ ]:
dc_centroids.shape  # matches our Graph.n

## Visualize

In [ ]:
flows = dc_flow_graph.adjacency.reset_index()

In [ ]:
flows

In [ ]:
# mapping of geoid to (point) geometry
dc.centroid.head()

In [ ]:
flows['focal'] = flows['focal'].replace(dc_centroids['geometry'].to_dict())
flows['neighbor'] = flows['neighbor'].replace(dc_centroids['geometry'].to_dict())

In [ ]:
flows

In [ ]:
origins = gpd.GeoSeries(flows.focal).get_coordinates().values

destinations = (
    gpd.GeoSeries(flows.neighbor).get_coordinates().values
)

In [ ]:
origins

In [ ]:
from shapely import LineString

In [ ]:
LineString([flows['focal'][0], flows['neighbor'][0]])

In [ ]:
lines = gpd.GeoSeries([LineString([row[1]['focal'], row[1]['neighbor']]) for row in flows.iterrows()], crs=4326)

In [ ]:
flow_lines =gpd.GeoDataFrame(flows['weight'], geometry=lines)

In [ ]:
flow_lines

In [ ]:
lines.explore()

In [ ]:
viz(flow_lines)

In [ ]:
dc_flow_graph.adjacency

In [ ]:
flow_lines[['weight', 'geometry']]

In [ ]:
brushing_extension = BrushingExtension()
brushing_radius = 300

arc_layer = ArcLayer(
    table=flow_lines[['weight', 'geometry']].to_arrow(),
    get_width=flow_lines.weight.apply(lambda x: x**.7),
    get_source_position=origins,
    get_target_position=destinations,
    opacity=0.4,
    pickable=False,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
    get_source_color=get_color_array(flow_lines.weight**.7, scheme='fisher_jenks', k=10, cmap='inferno_r'),
    get_target_color=get_color_array(flow_lines.weight**.7, scheme='fisher_jenks', k=10, cmap='inferno_r')
)

source_gdf = gpd.GeoDataFrame(geometry=flows["focal"], crs=4326)
target_gdf = gpd.GeoDataFrame(geometry=flows["neighbor"], crs=4326)

tgt = ScatterplotLayer.from_geopandas(
    target_gdf,
    radius_scale=30,
    pickable=True,
    stroked=False,
    filled=True,
    line_width_min_pixels=2,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)

src = ScatterplotLayer.from_geopandas(
    source_gdf,
    radius_scale=15,
    pickable=False,
    stroked=False,
    filled=True,
    line_width_min_pixels=2,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
)

bounds = PolygonLayer.from_geopandas(
    dc,
    get_fill_color=[255, 255, 255, 200],
    stroked=True,
    line_width_min_pixels=0.5,
    pickable=False,
    opacity=0.3
)

ogb = ScatterplotLayer.from_geopandas(
    source_gdf,
    get_fill_color=[255, 255, 255, 200],
    stroked=True,
    line_width_min_pixels=0.5,
    pickable=False,
)

gmap = Map(layers=[bounds, src, tgt, arc_layer,ogb], picking_radius=500)
gmap.to_html('dc_flow_orig_orient.html')


gmap

the mouseover is keyed on *home* (origin), so the visualiztion shows flows *away* from the selected location. To focus on destinations/inflows, we can reverse the orientation in the ArcLayer

In [ ]:
flows

In [ ]:
dest_graph = Graph.from_adjacency(dc_flows,
    focal_col="w_tr_geocode",
    neighbor_col="h_tr_geocode",
    weight_col="S000",
)

dest_graph_flows = dest_graph.adjacency.reset_index()

dest_graph_flows['focal'] = dest_graph_flows['focal'].replace(dc_centroids['geometry'].to_dict())
dest_graph_flows['neighbor'] = dest_graph_flows['neighbor'].replace(dc_centroids['geometry'].to_dict())

dest_lines = gpd.GeoSeries([LineString([row[1]['focal'], row[1]['neighbor']]) for row in dest_graph_flows.iterrows()], crs=4326)
dest_lines =gpd.GeoDataFrame(dest_graph_flows['weight'], geometry=dest_lines)

origins = gpd.GeoSeries(dest_graph_flows.focal).get_coordinates().values

destinations = (
    gpd.GeoSeries(dest_graph_flows.neighbor).get_coordinates().values
)

arc_layer = ArcLayer(
    table=dest_lines[['weight', 'geometry']].to_arrow(),
    get_width=dest_lines.weight.apply(lambda x: x**.7),
    get_source_position=origins,
    get_target_position=destinations,
    opacity=0.4,
    pickable=False,
    extensions=[brushing_extension],
    brushing_radius=brushing_radius,
    get_source_color=get_color_array(dest_lines.weight**.8, scheme='fisher_jenks', k=10, cmap='inferno_r'),
    get_target_color=get_color_array(dest_lines.weight**.6, scheme='fisher_jenks', k=10, cmap='inferno_r')
)
gmap = Map(layers=[bounds, src, tgt, arc_layer,ogb], picking_radius=500)

gmap.to_html('dc_flow_dest_orient.html')

gmap

In [ ]:
dc = dc.set_index("geoid")

## Migration as a Graph

the matrix representation of the network shows us the level of flow from each origin-destination pair

### Matrix Representation

In [ ]:
dc_flow_graph.sparse.todense()

there are lots of "internal flows", i.e. commutes that begin and end in the same tract (relatively large numbers along the diagonal). But it's common in migration studies to focus on inter-zonal flows, so we remove these--but first we will record the total numbers

In [ ]:
all_commutes = pd.Series(
    dc_flow_graph.sparse.sum(axis=1), index=dc_flow_graph.unique_ids, name="flows"
)

now save the total number of internal commutes (self-loops in the network; the diagonal of the matrix)

In [ ]:
# how many self-loops (trips originating and ending in the same tract
# not sure if there's an easier way to get this, but its the diagonal of the weights matrix
intra_commutes = pd.Series(
    np.diag(dc_flow_graph.sparse.todense()),
    index=dc_flow_graph.unique_ids,
    name="self_loops",
)
intra_share = intra_commutes / all_commutes

In [ ]:
intra_commutes

just so we can take a look: internal commutes are a small but non-significant share of our job flow data

In [ ]:
intra_share

In [ ]:
intra_commutes.sum() / all_commutes.sum()

in DC, 3.6% of the employees in LODES data have a job and workplace in the same census tract

In [ ]:
dc.join(intra_commutes, how="left").explore(
    "self_loops",
    style_kwds={"weight": 0.5},
    tiles="cartodb positron",
    tooltip="self_loops",
    scheme="fisherjenks",
    cmap="RdBu_r",
)

In [ ]:
dc.join(intra_share.rename("intra_share"), how="left").explore(
    "intra_share",
    style_kwds={"weight": 0.5},
    tiles="cartodb positron",
    tooltip="intra_share",
    scheme="fisherjenks",
    cmap="RdBu_r",
)

### Network Representation

to look at directionality (sending regions vs receiving regions) we can use network method from `networkx`

we only want to consider incoming and outgoing flows, so we first remove the diagonal in our `Graph` (and drop those observations), then convert to a `networkx` object

In [ ]:
dc_nx = dc_flow_graph.assign_self_weight(0).eliminate_zeros().to_networkx()

the *degree* of a node is the number of connections it has. Out-degree is the number of outgoing connections and in-degree is the number of incoming connections. For a focal observation, the *out degree* is the number of 'neighbors' it has (in spatial graph terminology), which is the same as its *cardinality*

In [ ]:
dc_flow_graph.assign_self_weight(0).eliminate_zeros().cardinalities

In [ ]:
pd.Series(dict(dc_nx.out_degree))

when the degree measure is weighted, each link/edge/relationship is multiplied by by its weight. Thus the weighted degree for each node is the sum of its weights  (in spatial econometric parlance, this is equivalent to an un-transformed *spatial lag*)

with out flow data, the weighted out degree measures the number of trips moving away from a home tract in the AM, and in-degree measures the number of trips incoming during the AM commute. The weighted degree (plain) is their sum, capturing the total fluctuation in the unit

In [ ]:
out_degree = pd.Series(
    dict(dc_nx.out_degree(dc_nx, weight="weight")), name="out_degree"
)
in_degree = pd.Series(
    dict(dc_nx.in_degree(dc_nx, weight="weight")), name="in_degree"
)

degree = pd.Series(dict(dc_nx.degree(dc_nx, weight="weight")), name="degree")

note the weighted out-degree is the same as the *spatial lag* when the weights matrix is unstandardized

In [ ]:
lag = dc_flow_graph.lag(np.ones(dc_flow_graph.n))

In [ ]:
pd.Series(lag).hist()

In [ ]:
out_degree.hist()

In [ ]:
pop_change = intra_commutes + in_degree - out_degree

In [ ]:
pop_change.hist()

In [ ]:
# daily population change
# internal movers plus immigration minus emigration
intra_commutes + in_degree - out_degree

In [ ]:
out_degree

In [ ]:
out_degree.sum() + intra_commutes.sum() == all_commutes.sum()

In [ ]:
dc_flow_graph.adjacency.reset_index().groupby("neighbor").sum().sum()

In [ ]:
out_degree.sum()

In [ ]:
out_degree.sum() / all_commutes.sum()

In [ ]:
all_commutes.sum()

In [ ]:
import seaborn as sns
sns.heatmap(dc_flow_graph.sparse.todense())

In [ ]:
out_degree.hist(bins=20)

In [ ]:
in_degree.sum()

In [ ]:
in_degree.hist(bins=20)

In [ ]:
dc.join(out_degree, how="left").explore(
    "out_degree",
    style_kwds={"weight": 0.5},
    tiles="cartodb positron",
    tooltip="out_degree",
    scheme="fisherjenks",
)

In [ ]:
dc.join(in_degree, how="left").explore(
    "in_degree",
    style_kwds={"weight": 0.5},
    linewidth=0.5,
    tiles="cartodb positron",
    tooltip="in_degree",
    scheme="fisherjenks",
)

In [ ]:
dc.join((out_degree / all_commutes).rename("out_share"), how="left").plot(
    "out_share",
    linewidth=0.5,
    #style_kwds={"weight": 0.5},
    #tiles="cartodb positron",
    #tooltip="out_share",
    scheme="fisherjenks",
    cmap="RdBu_r",
)

in-migration as a share of all change

In [ ]:
(in_degree / (intra_commutes + in_degree)).hist()

In [ ]:
dc.join(
    (in_degree / (intra_commutes + in_degree)).rename("in_share"), how="left"
).plot(
    "in_share",
    #style_kwds={"weight": 0.5},
    #tiles="cartodb positron",
    #tooltip="in_share",
    linewidth=0.5,
    scheme="fisherjenks",
    cmap="RdBu_r",
)

In [ ]:
(out_degree / all_commutes).hist()

In [ ]:
dc.join(((in_degree - out_degree) / degree).rename("diff"), how="left").plot(
    "diff",
    #style_kwds={"weight": 0.5},
    #tiles="cartodb positron",
    #tooltip="diff",
    linewidth=0.5,
    scheme="fisherjenks",
    cmap="RdBu_r",
    legend=True
)

In [ ]:
out_degree / all_commutes

## Spatial Interaction Models

Spatial interaction models are a classic regional science technique for modeling
flows (migration, commuting, trading, etc). Spatial interaction models try to
explain these flows between origin and destination as a function of origin
(push) and destination (pull) factors, attenuated by the cost of traveling
between them. The PySAL `spint` package implements a custom estimation engine
designed for fast performance on sparse design matrices (like those typical in
spatial interaction models), but they can also be fit using conventional
statistical packages like `statsmodels` albeit with a performance downgrade.

In this model, the dependent variable $y$ is the level of flow between an origin
and destination. Our `Graph` is based on flows, which means $y$ is the value of
the "weight" held in `balt_flow_graph`, thus can use the adjacency list as our
core data structure.

> Describing spatial interaction with an equation has a long and storied
> history, with Carrothers (1956), Sen and Smith (1995), and Farmer and Oshan
> (2017), among many others, furnishing point-in-time reviews of this topic in
> terms of its popular gravity model formulation, whose general form is as
> follows: 
 
$$F_{ij} = g[O_i, D_j, f(d_{ij})]$$

> where $F_{ij} denotes the flow between origin $i$ and destination $j$ areal
> units, $O_i$ is some attribute variable(s) quantifying relevant features of
> origin $i$, $D_j$ is some attribute variable(s) quantifying relevant features
> of destination $j$, $g$ is some function (frequently multiplicative in form),
> and $f(d_{ij})$ is some deterrent. The initial functional form resulted in Eq.
> (7.1) being transformed to a log-linear version and a normally distributed
> error attached to it for estimation and inference purposes, yielding 

$$ log(F_{ij} + \delta) = log(\kappa) + \alpha log(O_i)  + \beta log(D_j) + \gamma d_{ij} + \epsilon_{ij} $$

> where `log` denotes natural logarithm, $\epsilon_{ij}$ is a normally
> distributed random error term, $\delta \geq 0$ added to the response variable
> $F_{ij}$ is a translation parameter needed if any $F_{ij} = 0$, and $\kappa, \alpha, \beta$ and $\gamma$ are linear regression coefficients; sometimes this
> equation includes $log(d_{ij})$ rather than $d_{ij}$. This specification
> results in exponents for the $O_i$ and $D_j$ variables when the equation is
> backtransformed to its multiplicative form. Difficulties associated with this
> implementation include that $F_{ij}$ often is a count for which many cases are
> zero (hence the need for $\delta$).
>
> --- @griffith2019SpatialInteraction

@getis1991SpatialInteraction @putman1989EffectsSpatial @griffith2019SpatialInteraction @fotheringham1989SpatialInteraction

(as usual, there's an insane Tobler paper on the topic [@tobler1983AlternativeFormulation])

In [ ]:
dc_flow_graph = dc_flow_graph.assign_self_weight(0).eliminate_zeros()

In [ ]:
# sparse representation
dc_interaction = (
    dc_flow_graph.assign_self_weight(0).eliminate_zeros().adjacency.reset_index()
)
dc_interaction

In [ ]:
# for our dataset we want the full dense matrix
dc_interaction = pd.Series(dc_flow_graph.sparse.toarray().reshape(-1), 
                             index=pd.MultiIndex.from_product([dc_flow_graph.unique_ids, 
                                                               dc_flow_graph.unique_ids.rename('neighbor')])).rename('weight')

In [ ]:
dc_interaction = dc_interaction.reset_index()
dc_interaction

in spatial interaction terms focal=origin and neighbor=destination, so we can attach attributes of origins and destinations by doing left-joins on the y-variable successively based on focal, then neighbor

In [ ]:
# first merge origin attributes
dc_interaction = dc_interaction.merge(
    dc.drop(columns=["geometry"]), left_on="focal", right_index=True, how="left"
)

In [ ]:
# now merge destination attributes
dc_interaction = dc_interaction.merge(
    dc.drop(columns=["geometry"]),
    left_on="neighbor",
    right_index=True,
    how="left",
    suffixes=["_origin", "_destination"],
)

note the suffixes differentiate columns referring to origin and destination values, e.g. `median_household_income_origin` and `median_household_income_destination`

In [ ]:
dc_interaction

In [ ]:
dc_interaction[
    ["weight", "median_household_income_origin", "median_household_income_destination"]
]

for a spatial interaction model we also need *another* Graph that measures the distance between observations. There are many ways to do this, but the easiest is to use a distance band weight where all observations are guaranteed to lie within the threshold (giving us a pairwise matrix)

In [ ]:
dc = dc.to_crs(dc.estimate_utm_crs())

keep only tracts in the dataframe in our flow graph (origins), then get distance between observations with no decay

In [ ]:
dc = dc[dc.index.isin(dc_flow_graph.unique_ids)]
dc_dist = Graph.build_distance_band(
    dc.set_geometry(dc.centroid), threshold=1e20, binary=False, alpha=1
)

In [ ]:
dc_dist.summary()

In [ ]:
dc_dist.adjacency

In [ ]:
# subset the distance graph by the travel graph (remove destinations we dont need)
# but this resets weights to 1
dc_dist_adj = dc_dist.intersection(dc_flow_graph).adjacency

In [ ]:
# update with the old values
dc_dist_adj.update(dc_dist.adjacency)

In [ ]:
dc_interaction["distance"] = dc_dist.sparse.toarray().reshape(-1)

In [ ]:
dc_interaction['weight'] = dc_interaction['weight'].astype(int)

finally, we'll do some simple cleanup since we have missing values etc. 

In [ ]:
# mean-impute missing values
dc_interaction = dc_interaction.fillna(dc_interaction.mean(numeric_only=True))

# increment cols by 1 to make logs easier
dc_interaction['distance'] = dc_interaction['distance'] +1

dc_interaction['n_total_pop_origin'] = dc_interaction['n_total_pop_origin'] +1
dc_interaction['n_total__pop_destination'] = dc_interaction['n_total_pop_destination'] +1

dc_interaction['p_nonhisp_black_persons_origin'] = dc_interaction['p_nonhisp_black_persons_origin'] +1
dc_interaction['p_nonhisp_black_persons_destination'] = dc_interaction['p_nonhisp_black_persons_destination'] +1

dc_interaction = dc_interaction.copy()

In [ ]:
dc_interaction

## Using `spint` and `statsmodels`

@liao2025DataDrivenApproach, @oshan2016PrimerWorking, @oshan2021SpatialStructure, @oshan2025GeneralizedAdditive, @rey2021PySALEcosystem, @wilson1967StatisticalTheory

see @oshan2016PrimerWorking for a deeper discussion of spatial interaction models and their implementation details in PySAL

### Gravity

In [ ]:
g = Gravity(
    dc_interaction["weight"].values,
    o_vars=dc_interaction[["n_total_pop_origin", "median_household_income_origin", "p_nonhisp_black_persons_origin"]].values,
    d_vars=dc_interaction[
        ["n_total_pop_destination", "median_household_income_destination",  "p_nonhisp_black_persons_destination"]
    ]
    .values,
    cost=dc_interaction["distance"],
    cost_func="exp",
    constant=True,
).fit()

In [ ]:
g.params

In [ ]:
g.adj_pseudoR2

In [ ]:
dc_interaction["distance"]

In [ ]:
g_statsmodels = smf.glm(
    "weight ~ 1 + np.log(n_total_pop_origin) + np.log(median_household_income_origin) + np.log(p_nonhisp_black_persons_origin) + np.log(n_total_pop_destination) + np.log(median_household_income_destination) + np.log(p_nonhisp_black_persons_destination) + distance",
    family=families.Poisson(),
    data=dc_interaction,).fit(cov_type='HC0', scale='X2')

In [ ]:
g_statsmodels.summary()

In [ ]:
g.params.round(4)

In [ ]:
out = g_statsmodels.params.rename("statsmdodels").to_frame().copy()
out["spint"] = g.params.T.round(4)

compare coefs in a table format

In [ ]:
out

### Production-Constrained

unit-level fixed effects for origins

In [ ]:
mod_prod = Production(
    dc_interaction["weight"].values,
    dc_interaction["focal"].values,
    d_vars=dc_interaction[
        ["n_total_pop_destination", "median_household_income_destination", "p_nonhisp_black_persons_destination"]
    ]
    .values,
    cost=dc_interaction["distance"].values,
    cost_func="exp",
).fit()

In [ ]:
mod_prod.params[-4:]

In [ ]:
mod_prod.adj_pseudoR2

also do the power version/

In [ ]:
mod_prod_pow = Production(
    dc_interaction["weight"].values,
    dc_interaction["focal"].values,
    d_vars=dc_interaction[
        ["n_total_pop_destination", "median_household_income_destination", "p_nonhisp_black_persons_destination"]
    ]
    .values,
    cost=dc_interaction["distance"].values,
    cost_func="pow",
).fit()

In [ ]:
mod_prod_pow.params[-4:]

In [ ]:
mod_prod_pow.adj_pseudoR2

### Attraction-Constrained

unit-level fixed effects for destinations

In [ ]:
mod_attr = Attraction(
    flows=dc_interaction["weight"].values,
    destinations=dc_interaction["neighbor"].values,
    o_vars=dc_interaction[["n_total_pop_origin", "median_household_income_origin",  "p_nonhisp_black_persons_origin"]]
    .values,
    cost=dc_interaction["distance"].values,
    cost_func="exp",
)

the constrained models can be calibrated *locally*, i.e. each observation has its own set of coefficients, thus we can map the effect of a given parameter, say distance:

In [ ]:
attr_local = pd.DataFrame(mod_attr.local())

In [ ]:
attr_local.columns

In [ ]:
dc.assign(dist=attr_local['param3'].values).explore('dist', scheme='quantiles', tooltip='dist',  cmap='RdBu_r',   style_kwds={"weight": 0.5},
 tiles='cartodb positron')

In [ ]:
mod_attr.fit()

In [ ]:
mod_attr.params[-4:]

In [ ]:
mod_attr.pseudoR2

In [ ]:
# attraction-constrained
a_statsmodels = smf.glm(
    "weight ~ 1 + I(neighbor) + np.log(n_total_pop_origin) + np.log(median_household_income_origin) + np.log(p_nonhisp_black_persons_origin) + distance",
    family=families.Poisson(),
    data=dc_interaction
).fit()

In [ ]:
a_statsmodels.summary()

In [ ]:
mod_attr.pseudoR2

In [ ]:
mod_attr.params[-4:].round(4)

In [ ]:
a_statsmodels.params[-4:]

In [ ]:
# mcfadden
1 - (a_statsmodels.llf / a_statsmodels.llnull)

In [ ]:
# cohen
(a_statsmodels.null_deviance - a_statsmodels.deviance) / a_statsmodels.null_deviance

In [ ]:
# CS
1 - np.exp((2 / a_statsmodels.nobs) * (a_statsmodels.llnull - a_statsmodels.llf))

In [ ]:
a_statsmodels.pseudo_rsquared()

yep, the CS psueudo $R^2$ matches the reported one

### Doubly-Constrained

fixed effects for both origin and destination

In [ ]:
d = Doubly(
    dc_interaction["weight"].values,
    dc_interaction["focal"].values,
    dc_interaction["neighbor"].values,
    cost=dc_interaction["distance"],
    cost_func="exp",
).fit()

In [ ]:
d.adj_pseudoR2

In [ ]:
d.params[-1:].round(5)

In [ ]:
d_statsmodels = smf.glm(
    "weight ~ 1 + C(focal) + C(neighbor) + distance",
    family=families.Poisson(),
    data=dc_interaction
).fit()

In [ ]:
d_statsmodels.summary()

In [ ]:
d.params[-1:]

In [ ]:
d_statsmodels.params[-1]